In [ ]:
import re

import pandas as pd

KEY_COL = "key"


def process_name(df, col):
    df[col] = (df[col]
               .apply(lambda s: s.replace("\n", " "))
               # .apply(lambda s: s.replace('"', ""))
               .apply(lambda s: re.sub('\"|\“', '', s))
               .apply(str.strip)
               )

    df[KEY_COL] = (df[col]
                   .apply(str.lower)
                   .apply(lambda s: " ".join((word
                                              for word in s.split()
                                              if word
                                              if re.match(r"\w+",word)
                                              if word not in (
                                                  "оод",
                                                  "еоод",
                                                  "ад",
                                                  "ауто",
                                                  "авто",
                                                  "кар",
                                                  "мото",
                                                  "българия",
                                                  "софия",
                                                  "сервиз",
                                                  "сервис",
                                                  "сървиз",
                                                  "сървис",
                                                  "колор",
                                                  "пейнт",
                                                  "груп",
                                                  "и",
                                                  "енд",
                                              ))))
                   )

    return df

In [ ]:
asset_garages = pd.read_csv("Asset_garages_Sofia.csv",
                            skiprows=1,
                            )
asset_garages = asset_garages[asset_garages['Unnamed: 0'].notna()]
asset_garages = asset_garages[asset_garages['Сервиз'].notna()]
asset_garages = process_name(asset_garages, "Сервиз")
asset_garages.to_csv("Asset_garages_Sofia_proc.csv")
# set(asset_garages[KEY_COL])
asset_garages


In [ ]:
dzi_garages = pd.read_csv("DZI-Garages-Sept20.csv")
dzi_garages = dzi_garages[dzi_garages["Име на Сервиз"].notna()]
dzi_garages = process_name(dzi_garages, "Име на Сервиз")
dzi_garages.to_csv("DZI_Garages_Sept20_proc.csv")
# set(dzi_garages[KEY_COL])
dzi_garages

In [ ]:
dzi_sofia = dzi_garages[dzi_garages.apply(
    lambda x: "софия" in x['Град'].lower(),
    axis=1,
)]
dzi_sofia

In [ ]:
set(asset_garages[KEY_COL]).intersection(set(dzi_sofia[KEY_COL]))

In [ ]:
garage_match = pd.merge(asset_garages,dzi_sofia,on=KEY_COL)
garage_match.to_csv("sofia_garages.csv")
garage_match